In [1]:
import os
print(os.getcwd())

/mnt/c/Users/Komar/Lab/retroparty


In [2]:
os.chdir('/mnt/c/Users/Komar/Lab')
print(os.getcwd())

/mnt/c/Users/Komar/Lab


### Output data filtration

In [3]:
DIR = os.getcwd() + '/OutPutDir/strong_settings/'

In [4]:
# TO DO
# make list of file in directory 
file_names = ['SRR7624520+test__', 'SRR7624521+test__', 'SRR7624522+test__', 'SRR7624523+test__']

In [83]:
import pandas as pd
res = pd.read_table(DIR + '/metatable_ts.txt')
print(res.shape)
res.head()

(20494, 38)


,METACLUSTER_ID,FILENAME,READNAME,CHR,POS,INS_STRAND,PRIMER,RE,RE_AMOUNT,RE_HAMMING,...,SRR7624520+test___NUM_BC,SRR7624521+test___NUM_BC,SRR7624522+test___NUM_BC,SRR7624523+test___NUM_BC,SRR7624520+test___NUM_TLEN,SRR7624521+test___NUM_TLEN,SRR7624522+test___NUM_TLEN,SRR7624523+test___NUM_TLEN,MDFLAG_TEMPLATE,MDMATCH
0,1,SRR7624522+test__,SRR7624522.6008022.2,chr1,147922,+,GAGCCACCGCGC,CCGGCC,1,0,...,0,0,1,0,0,0,1,0,MD:Z:32,32
1,2,SRR7624521+test__,ARTIFICIAL:2262337645,chr1,196273,+,GAGCCACCGCGC,CCGGCC,4,0,...,1,1,1,1,1,1,1,1,*,0
2,3,SRR7624522+test__,SRR7624522.6084773.2,chr1,1549246,+,GAGCCACCGCGC,CCGGCC,4,0,...,1,1,1,1,1,1,1,1,MD:Z:22A9,31
3,4,SRR7624521+test__,SRR7624521.6995156.1,chr1,1934731,+,GAGCCACCGCGC,CCGGCC,1,0,...,0,1,0,0,0,1,0,0,MD:Z:9G18,27
4,5,SRR7624521+test__,SRR7624521.5871642.1,chr1,1995737,+,GAGCCACCGCGC,CCGGCC,1,0,...,0,1,0,0,0,1,0,0,MD:Z:32,32


In [104]:
res.columns

Index(['METACLUSTER_ID', 'FILENAME', 'READNAME', 'CHR', 'POS', 'INS_STRAND',
       'PRIMER', 'RE', 'RE_AMOUNT', 'RE_HAMMING', 'R1', 'R2', 'TLEN',
       'CIGAR_R1', 'MDFLAG_R1', 'MD_SUM', 'MISMATCH', 'INSERTION', 'DELETION',
       'R_SITE', 'MISS_P_HAMMING', 'MISS_RE_HAMMING', 'REPEAT_FUNC',
       'REPEAT_NAME', 'SRR7624520+test___NUM_READS',
       'SRR7624521+test___NUM_READS', 'SRR7624522+test___NUM_READS',
       'SRR7624523+test___NUM_READS', 'SRR7624520+test___NUM_BC',
       'SRR7624521+test___NUM_BC', 'SRR7624522+test___NUM_BC',
       'SRR7624523+test___NUM_BC', 'SRR7624520+test___NUM_TLEN',
       'SRR7624521+test___NUM_TLEN', 'SRR7624522+test___NUM_TLEN',
       'SRR7624523+test___NUM_TLEN', 'MDFLAG_TEMPLATE', 'MDMATCH'],
      dtype='object')

In [84]:
# checking for null occurrences 
df_0 = res.copy(deep=True)
for i in range(len(file_names)):
    df_0 = df_0[df_0[file_names[i] + '_NUM_READS'] == 0]
print(df_0.shape[0])
#df_0.head()

0


In [85]:
# not somatic (germline) insertions (exists in all samples)
df_germ = res.copy(deep=True)
for i in range(len(file_names)):
    df_germ = df_germ[df_germ[file_names[i] + '_NUM_READS'] > 0]
print('all:', df_germ.shape[0],  '  including test:', df_germ[df_germ['READNAME'].str.contains('ARTIFICIAL')].shape[0])
#df_germ.head()

all: 2941   including test: 877


In [86]:
# somatic and quasi-somatic (exists for example in 3/4 samples) insertions
df_qsom = res.copy(deep=True)
df_qsom = df_qsom.drop(df_germ.index, axis=0)
print(df_qsom.shape, '  including test:', df_qsom[df_qsom['READNAME'].str.contains('ARTIFICIAL')].shape[0])
#df_qsom.head()

(17553, 38)   including test: 74


In [87]:
# insertions specific only for this tissue/sample(file) 
df_somatic_list = []
overall_num = 0
for i in range(len(file_names)):
    df = df_qsom.copy(deep=True)
    df = df[df[file_names[i] + '_NUM_READS'] > 0]
    for j in range(1, len(file_names)):
        df = df[df[file_names[i - j] + '_NUM_READS'] == 0]
    print(file_names[i], df.shape[0], '  test:', df[df['READNAME'].str.contains('ARTIFICIAL')].shape[0])
    overall_num += df.shape[0]
    df_somatic_list.append(df.copy(deep=True))
print(overall_num)

SRR7624520+test__ 2480   test: 1
SRR7624521+test__ 5886   test: 1
SRR7624522+test__ 3046   test: 0
SRR7624523+test__ 2171   test: 0
13583


In [151]:
def complement(char):
    if char == 'A':
        return 'T'
    if char == 'T':
        return 'A'
    if char == 'C':
        return 'G'
    if char == 'G':
        return 'C'
    if char == 'N':
        return 'N'

def reverse_comp(seq):
    seq_ls = list(seq)
    new_seq = []
    for i in reversed(seq_ls):
        new_seq.append(complement(i))
    return "".join(new_seq)

In [122]:
# preparing for copying into blat 
from tqdm import trange
for i in trange(len(file_names)):
    # filtration
    df_tmp = df_somatic_list[i].copy(deep=True)
    df_tmp = df_tmp[df_tmp[file_names[i] + '_NUM_READS'] > 1].copy(deep=True)
    df_tmp = df_tmp[df_tmp['R1'].str.len() > 25].copy(deep=True)  # because Blat can aling seq > 25nt
    df_tmp = df_tmp[df_tmp['R2'].str.len() > 25].copy(deep=True)
    df_tmp = df_tmp[df_tmp['MDMATCH'] < 32].copy(deep=True)  # template_switch_filtration: 12 primer + 6 dALU + 14 flank = 32
    print(file_names[i], ' was:', df_somatic_list[i].shape[0], ' become:', df_tmp.shape[0])

    # make R2 reverse complement
    df_tmp['R2'] = df_tmp['R2'].apply(reverse_comp)

    # writing reads into files
    df_tmp.to_csv(f'{file_names[i]}_filtered.csv')
    print(f'{file_names[i]} reads were written')

100%|██████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 30.84it/s]

SRR7624520+test__  was: 2480  become: 130
SRR7624520+test__ reads were written
SRR7624521+test__  was: 5886  become: 524
SRR7624521+test__ reads were written
SRR7624522+test__  was: 3046  become: 127
SRR7624522+test__ reads were written
SRR7624523+test__  was: 2171  become: 73
SRR7624523+test__ reads were written


### UCSC Blat parsing

In [124]:
from tqdm import trange
import pandas as pd

filtered_flanks = []
for i in trange(len(file_names)):
    df_tmp = pd.read_csv(f'{file_names[i]}_filtered.csv')
    filtered_flanks.append(df_tmp.copy(deep=True))

100%|██████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 62.53it/s]


In [145]:
def get_reads_fa(reads_list, reads_names, R_n=1):
    line = ''
    for read, name in zip(reads_list, reads_names):
        line = line + '>' + name + f'_R{R_n}' + '\n' + read + '\n'
    return line

In [153]:
%%time
import pandas as pd
from tqdm import trange
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.select import Select

service = Service(executable_path="/usr/bin/chromedriver")
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(service=service, options=chrome_options)

# go to the UCSC Blat
driver.get("https://genome.ucsc.edu/cgi-bin/hgBlat?hgsid=1711647570_KRW9QfFHAHaje8cp8sWT4EtpGupE&command=start")
print(driver.title)

# set Hg38
selects = driver.find_elements(By.TAG_NAME, "select")
for i in selects:
    if i.get_property("name") == "db":
        element = i
        break
dropdown = Select(element)
dropdown.select_by_visible_text("Dec. 2013 (GRCh38/hg38)")

for i in trange(len(filtered_flanks)):
    for k in [1, 2]:
        print('submitting')    
        # set flank coords and submit
        range_input = driver.find_element(By.NAME, "userSeq")
        range_input.clear()
        #TO DO: push only 20 reads at once
        range_input.send_keys(get_reads_fa(filtered_flanks[i][f'R{k}'].tolist(), filtered_flanks[i]['READNAME'].tolist(), R_n=k))  # push all reads at once
        
        button = driver.find_element(By.NAME, "Submit")
        button.click()
        # get results 
        try:
            print('waiting')
            element = WebDriverWait(driver, 120).until(
                                EC.presence_of_element_located((By.TAG_NAME, "pre")) 
                                )
            print('writing')
            with open(f'{file_names[i]}_R{k}_blat_out_tmp.txt', 'w') as f:
                f.write(element.text)
            print('res written')
        except:
            print(f'exception for {file_names[i]}')
        # get back to request form       
        driver.back()

driver.quit()

Human BLAT Search


  0%|                                                                                                      | 0/4 [00:00<?, ?it/s]

wait
got res
wait


 25%|███████████████████████▌                                                                      | 1/4 [00:35<01:46, 35.41s/it]

got res
wait
got res
wait


...

got res
wait
got res
wait


 75%|█████████████████████████████████████████████████████████████████████▊                       | 3/4 [09:14<02:59, 179.67s/it]

got res
wait
got res
wait


100%|█████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [09:38<00:00, 144.64s/it]

got res
CPU times: user 1.48 s, sys: 98.7 ms, total: 1.58 s
Wall time: 9min 41s


In [155]:
for i in trange(len(filtered_flanks)):
    for k in [1, 2]:
        blat_out = pd.DataFrame(columns=['QUERY', 'SCORE', 'QSTART', 'QEND', 'QSIZE', 'IDENTITY', 'CHROM', 'STRAND', 'START', 'END', 'SPAN'])
        with open(f'{file_names[i]}_R{k}_blat_out_tmp.txt', 'r') as f:
            output = f.read()
        output = output.split('\n')
        for line in output:
                if line.count('browser'):
                    tmp = line.split(' ')
                    tmp = ' '.join(tmp).split()
                    blat_out.loc[len(blat_out.index)] = [tmp[2], tmp[3], tmp[4], tmp[5], tmp[6], tmp[7], tmp[8], tmp[9], tmp[10], tmp[11], tmp[12]]
        blat_out.to_csv(f'{file_names[i]}_R{k}_blat_out.csv') 

100%|██████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:38<00:00,  9.61s/it]


### Blat analysis

In [159]:
def hamming_dist(x1, x2):
    j = 0
    for i in range(len(x1)):
        if x1[i] != x2[i]:
            j = j + 1
    return j

def find_overlap(str1, str2):
    if len(str1) < len(str2):
        str_tmp = str1
        str1 = str2
        str2 = str_tmp
    if len(str1) >= len(str2):
        delta = len(str1) - len(str2)
        for i in range(len(str1)):
            if delta - i >= 0:
                part1 = str1[i:len(str2) + i]
                part2 = str2
            else:
                part1 = str1[i:]
                part2 = str2[0:len(str2) + delta - i]
            # print(part1, part2)
            if hamming_dist(part1, part2) == 0:
                #print(part1, len(part1))
                return part1
        return 'no overlap'

In [43]:
import pandas as pd
from tqdm import trange

def format_coords(df_row, expansion=0):
    low_coord = int(min(df_row['START_1'], df_row['START_2'], df_row['END_1'], df_row['END_2'])) - expansion
    high_coord = int(max(df_row['START_1'], df_row['START_2'], df_row['END_1'], df_row['END_2'])) + expansion
    return f"{df_row['CHROM_1']}:{low_coord}-{high_coord}"

def blat_merge_and_filtration(filename, expansion=0):
    df1 = pd.read_table(filename + '_R1_blat_out.csv', sep=',')
    df1 = df1.drop(['Unnamed: 0'], axis=1)
    
    df2 = pd.read_table(filename + '_R2_blat_out.csv', sep=',')
    df2 = df2.drop(['Unnamed: 0'], axis=1)
    
    read_names = df1['QUERY'].unique()
    merget_df = pd.DataFrame(columns=['QUERY', 
                                      'SCORE_1', 'QSIZE_1', 'QSTART_1', 'QEND_1', 'IDENTITY_1', 'CHROM_1', 'STRAND_1', 'START_1', 'END_1', 'SPAN',
                                      'SCORE_2', 'QSIZE_2', 'QSTART_2', 'QEND_2', 'IDENTITY_2', 'CHROM_2', 'STRAND_2', 'START_2', 'END_2', 'SPAN'
                                     ]
                            )
    for name in read_names:
        df1_1 = df1[df1['QUERY'].str.contains(name)].copy(deep=True)
        df2_1 = df2[df2['QUERY'].str.contains(name[:-3])].copy(deep=True)
        for i, row_1 in df1_1.iterrows():
            tmp_df = df2_1[df2_1['CHROM'] == row_1['CHROM']]
            for i, row_tmp in tmp_df.iterrows():
                if abs(row_tmp['START'] - row_1['START']) <= 1000 and abs(row_tmp['END'] - row_1['END']) <= 1000:
                    merget_df.loc[len(merget_df.index)] = [row_1['QUERY'][:-3], 
                                                           row_1['SCORE'], row_1['QSIZE'], row_1['QSTART'], row_1['QEND'], row_1['IDENTITY'],
                                                           row_1['CHROM'], row_1['STRAND'], row_1['START'], row_1['END'], row_1['SPAN'],
                                                           row_tmp['SCORE'], row_tmp['QSIZE'], row_tmp['QSTART'], row_tmp['QEND'], row_tmp['IDENTITY'], 
                                                           row_tmp['CHROM'], row_tmp['STRAND'], row_tmp['START'], row_tmp['END'], row_tmp['SPAN'] 
                                                          ]
    
    #print(name[:-3], df1_1.shape[0], df2_1.shape[0], merget_df.shape[0])
    merget_df.drop(merget_df[merget_df['CHROM_1'].str.contains('alt')].index, axis=0, inplace=True)
    merget_df.drop(merget_df[merget_df['CHROM_2'].str.contains('chrUn')].index, axis=0, inplace=True)
    merget_df = merget_df[merget_df['STRAND_1'] == merget_df['STRAND_2']]
    if merget_df.shape[0] > 1:
        merget_df = merget_df[merget_df['SCORE_1'] > 0.8 * merget_df['QSIZE_1']]
    if merget_df.shape[0] > 1:
        merget_df = merget_df[merget_df['SCORE_2'] > 0.8 * merget_df['QSIZE_2']]
    if merget_df.shape[0] > 1:
        merget_df = merget_df[merget_df['QSTART_1'] < 3]        
    if merget_df.shape[0] > 0:
        for i, row in merget_df.iterrows():
            merget_df.loc[i, f'FLANK_COORD+{expansion}'] = format_coords(row, expansion) 
    print(merget_df.groupby(['QUERY']).count()['SCORE_1'])
    return merget_df.copy(deep=True)

In [44]:
EXPANSION = 30

for i in trange(len(file_names)):
        df_tmp = blat_merge_and_filtration(file_names[i], EXPANSION)
        df_tmp.to_csv(f'{file_names[i]}_blat_filtred.csv')

 25%|███████████████████████▌                                                                      | 1/4 [00:03<00:09,  3.33s/it]

QUERY
SRR7624520.1085275.1      1
SRR7624520.1398826.1      1
SRR7624520.1706906.1    143
SRR7624520.2288684.1     48
SRR7624520.2297983.1      1
SRR7624520.2358731.1      2
SRR7624520.3416011.1    202
SRR7624520.3523217.1      1
SRR7624520.3813773.1      1
SRR7624520.4138350.1      2
SRR7624520.4707551.1      1
SRR7624520.5111332.1      1
SRR7624520.579996.1       1
SRR7624520.5900893.1      1
SRR7624520.6258397.1      1
SRR7624520.7322435.1      1
SRR7624520.7545356.1      1
SRR7624520.7954699.1      1
SRR7624520.8203499.1      1
SRR7624520.987629.1      78
Name: SCORE_1, dtype: int64


 50%|███████████████████████████████████████████████                                               | 2/4 [00:06<00:05,  2.98s/it]

QUERY
SRR7624521.10461099.1    35
SRR7624521.1091436.1      9
SRR7624521.1150382.1      1
SRR7624521.1177962.1      1
SRR7624521.1823484.1      2
SRR7624521.1832460.1      1
SRR7624521.1847393.1      2
SRR7624521.1966782.1      1
SRR7624521.2711306.1      9
SRR7624521.347999.1       1
SRR7624521.3782023.1      1
SRR7624521.4134439.1      1
SRR7624521.4152381.1      1
SRR7624521.471417.1       1
SRR7624521.5543254.1      6
SRR7624521.6035605.1      5
SRR7624521.6930550.1      4
SRR7624521.7186939.1      2
SRR7624521.926347.1      35
SRR7624521.9687035.1      1
Name: SCORE_1, dtype: int64


 75%|██████████████████████████████████████████████████████████████████████▌                       | 3/4 [00:08<00:02,  2.95s/it]

QUERY
SRR7624522.10034679.2    36
SRR7624522.11743518.2     1
SRR7624522.11895566.2    46
SRR7624522.1417982.2      1
SRR7624522.2253482.2      1
SRR7624522.2262301.2      1
SRR7624522.2360499.2      1
SRR7624522.4870103.2     23
SRR7624522.7357132.2      1
SRR7624522.8107189.2      1
SRR7624522.8791607.2     42
SRR7624522.9172898.2      1
SRR7624522.9325503.2      1
SRR7624522.999637.2       1
Name: SCORE_1, dtype: int64


100%|██████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:14<00:00,  3.59s/it]

QUERY
SRR7624523.1143445.2      1
SRR7624523.1884264.2     27
SRR7624523.2039583.2      1
SRR7624523.2337629.2      2
SRR7624523.2468953.2     34
SRR7624523.301840.2       1
SRR7624523.3366411.2     31
SRR7624523.410742.2       1
SRR7624523.4290877.2     17
SRR7624523.4338292.2    132
SRR7624523.4446101.2      1
SRR7624523.472562.2       1
SRR7624523.527837.2       1
SRR7624523.5450264.2     17
SRR7624523.5693166.2      1
SRR7624523.842490.2       1
SRR7624523.8560214.2     47
SRR7624523.925511.2       1
Name: SCORE_1, dtype: int64


### RepeatMas

In [39]:
from random import randint
def gen_rand_num_list(min, max, length):
    if length > max:
        return None
    rand_list = []
    for i in range(length): 
        rand_num = randint(min, max)
        while rand_list.count(rand_num) != 0:
            rand_num = randint(min, max)
        rand_list.append(rand_num) 
    return rand_list

In [55]:
import pandas as pd

blat_res = []
RM_res = []
for name in file_names:
    blat_res.append(pd.read_table(f'{name}_blat_filtred.csv', sep=','))
    RM_res.append(pd.DataFrame(columns=['QUERY', 'EXPANSION', 'FLANK_COORD', 'F_STRAND', 'REPEAT_NAME', 'REPEAT_COORD', 'R_STRAND', 'OVERLAPPING', 'UCSC_LINK', 
                                        'SCORE_1', 'CHROM_1', 'START_1', 'END_1', 'SCORE_2', 'CHROM_2', 'START_2', 'END_2', 
                                        'F_CHROM', 'F_START', 'F_END']))

In [56]:
for blat_res_part in blat_res:
    print('before', blat_res_part.shape[0])
    amount_list = blat_res_part.groupby(['QUERY']).count()['SCORE_1'].tolist()
    read_names = blat_res_part['QUERY'].unique()
    for num, name in zip(amount_list, read_names):
        if num > 10:
            rand_list = gen_rand_num_list(0, num-1, num-10)
            indexes = blat_res_part[blat_res_part['QUERY'].str.contains(name)].index.tolist()
            index_for_del = []
            for i in rand_list:
                index_for_del.append(indexes[i])
            blat_res_part.drop(index_for_del, axis=0, inplace=True)
    print('after', blat_res_part.shape[0])

before 489
after 58
before 119
after 69
before 157
after 50
before 317
after 82


In [ ]:
def calc_overlap(str:flank_coord, r_start, r_end):
    f_start = int(flank_coord.split(':')[1].split('-')[0])
    f_end = int(flank_coord.split(':')[1].split('-')[1])
    if r_start < f_start:
        if 
    else:
        
    return 

In [57]:
%%time
from tqdm import trange
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.select import Select

service = Service(executable_path="/usr/bin/chromedriver")
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(service=service, options=chrome_options)

# go to the RepeatMasker
driver.get("https://www.repeatmasker.org/cgi-bin/AnnotationRequest")
print(driver.title)

# set Hg38
try:
    element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.NAME, "genome"))
    )
    dropdown = Select(element)
    dropdown.select_by_visible_text("Human - Dec 2013 - hg38")
except:
    print("can't select genome")

# flank coords annotating
j = 0
for blat_res_part, RM_res_part in zip(blat_res, RM_res):
    for i in trange(blat_res_part.shape[0]):
        # set flank coords and submit
        try:
            range_input = driver.find_element(By.NAME, "range")
            range_input.clear()
            range_input.send_keys(blat_res_part.iloc[i][f'FLANK_COORD+{EXPANSION}'])
           
            button = driver.find_element(By.NAME, ".submit")
            button.click()
            error = ''
        except:
            print("can't pass coords")
    
        # get results 
        try:
            element = WebDriverWait(driver, 15).until(
                      EC.presence_of_element_located((By.TAG_NAME, "pre")) 
                      )
            link = element.find_element(By.TAG_NAME, "a").get_attribute("href")
            output = element.text
            output = output.split('\n')[2].split(' ')
            output = ' '.join(output).split()
            overlapping = calc_overlap(blat_res_part.iloc[i][f'FLANK_COORD+{EXPANSION}'], output[5], output[6])
            RM_res_part.loc[len(RM_res_part.index)] = [blat_res_part.iloc[i]['QUERY'], EXPANSION, blat_res_part.iloc[i][f'FLANK_COORD+{EXPANSION}'],
                                                       blat_res_part.iloc[i]['STRAND_1'], output[9], f"{output[4]}:{output[5]}-{output[6]}", 
                                                       output[8], 'overlapping', link, blat_res_part.iloc[i]['SCORE_1'], blat_res_part.iloc[i]['CHROM_1'],
                                                       blat_res_part.iloc[i]['START_1'], blat_res_part.iloc[i]['END_1'], blat_res_part.iloc[i]['SCORE_2'],
                                                       blat_res_part.iloc[i]['CHROM_2'], blat_res_part.iloc[i]['START_2'], blat_res_part.iloc[i]['END_2'],
                                                       output[4], output[5], output[6]
                                                      ]
        except:
            #print('no matching')
            RM_res_part.loc[len(RM_res_part.index)] = [blat_res_part.iloc[i]['QUERY'], EXPANSION, blat_res_part.iloc[i][f'FLANK_COORD+{EXPANSION}'],
                                                       blat_res_part.iloc[i]['STRAND_1'], '-', '-', 
                                                       '-', 0, '-', blat_res_part.iloc[i]['SCORE_1'], blat_res_part.iloc[i]['CHROM_1'],
                                                       blat_res_part.iloc[i]['START_1'], blat_res_part.iloc[i]['END_1'], blat_res_part.iloc[i]['SCORE_2'],
                                                       blat_res_part.iloc[i]['CHROM_2'], blat_res_part.iloc[i]['START_2'], blat_res_part.iloc[i]['END_2'],
                                                       '-', '-', '-'
                                                      ]
        # get back to request form
        driver.back()
    RM_res_part.to_csv(f'{file_names[j]}_RM_res.csv')
    j += 1


#quit from site
driver.quit()

SyntaxError: invalid syntax (<unknown>, line 57)

In [51]:
list_RM_tmp = RM_res.copy()

In [54]:
list_RM_tmp[0].head()

,flank coord,repeat coord,strand,repeat name,link UCSC
0,chr1:74835959-74836051,-,strand,-,-
1,chr10:43181695-43181830,chr10:43181632-43181790,+,FRAM,http://genome.ucsc.edu/cgi-bin/hgTracks?db=hg3...
2,chr10:24730036-24730128,chr10:24729962-24730232,+,AluSz,http://genome.ucsc.edu/cgi-bin/hgTracks?db=hg3...
3,chr3:98606795-98606886,chr3:98606642-98606958,C,AluSx1,http://genome.ucsc.edu/cgi-bin/hgTracks?db=hg3...
4,chr3:17464086-17464176,chr3:17463944-17464259,C,AluSp,http://genome.ucsc.edu/cgi-bin/hgTracks?db=hg3...
